# Project_1_Exploring Ebay Car Sales Data
the data set a classifieds section of the German eBay website. It has 50,000 data points and 20columns.

#### The data dictionary provided with data is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

In [ ]:
import pandas as pd
import numpy as np
autos = pd.read_csv('autos.csv',encoding='windows-1252')

In [ ]:
autos.info()
autos.head()
autos.isnull().sum()
print(autos.columns)

The column names use camelcase(camelCase) instead of Python's preferred snakecase(snake_case).
Thus, the following steps cleans the column names

In [ ]:
autos.columns = autos.columns.str.replace('PS','_ps')
autos.columns = autos.columns.str.replace('C','_c')
autos.columns = autos.columns.str.replace('T','_t')
autos.columns = autos.columns.str.replace('O','_o')
autos.columns = autos.columns.str.replace('R','_r')
autos.columns = autos.columns.str.replace('D','_d')
autos.columns = autos.columns.str.replace('P','_p')
autos.columns = autos.columns.str.replace('S','_s')
print(autos.columns)
autos.describe(include='all')

In [ ]:
autos.price = autos.price.str.replace('$','').str.replace(',','')
autos.odometer = autos.odometer.str.replace('km','').str.replace(',','')
autos[['price','odometer']] = autos[['price','odometer']].astype(int)
autos[['price','odometer']].head()
autos.rename({'odometer':'odometer_km'},inplace=True,axis=1)
autos.columns

In [ ]:
autos.price.unique().shape #2357 uinque values
autos['price'].describe() #min is 0, max is 10^8
autos.price.value_counts().sort_index(ascending=1).head(50)
autos = autos[autos.price.between(100,350001)]
autos.info()

In [ ]:
autos.odometer_km.unique().shape #13 unique values
autos.odometer_km.value_counts().sort_index() #seems no outliers
autos.odometer_km.describe() #min 5k, max 150k

In [ ]:
autos[['date_crawled','last_seen','date_created','month_of_registration','year_of_registration']].head()

#first 3 cols contains strings, the rest 2 contain int

In [ ]:
# the day: first 10 strings
autos['date_crawled'].str[:10].value_counts(normalize=1,dropna=0).sort_index(ascending=0)
autos['last_seen'].str[:10].value_counts(normalize=1,dropna=0).sort_index(ascending=0)
autos['date_created'].str[:10].value_counts(normalize=1,dropna=0).sort_index(ascending=0)

In [ ]:
autos.year_of_registration.describe() #min is 1000, max is 9999, must have outliers

In [ ]:
autos = autos[autos.year_of_registration.between(1900,2020)]# there are 48205 datapoints w/ registration year after 1900.
autos.year_of_registration.value_counts(normalize=1).sort_index() 

#### aggregation & analysis

In [ ]:
brands = autos['brand'].value_counts(normalize=True).head(6).index  #
mean_price = {}
for c in brands:
    select_rows = autos[autos['brand'] == c]
    mean = select_rows['price'].mean()
    mean_price[c] = mean
mean_price

In [ ]:
mean_mileage = {}
for c in brands:
    select_rows = autos[autos['brand'] == c]
    mean = select_rows['odometer_km'].mean()
    mean_mileage[c] = mean
mean_mileage

In [ ]:
mean_price_series = pd.Series(mean_price)
print(mean_price_series)
mean_mileage_series = pd.Series(mean_mileage)
df = pd.DataFrame(mean_price_series,columns=['mean_price'])
print(df)
df['mean_mileage'] = mean_mileage_series
df